In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime


now = datetime.now()

start = datetime(now.year - 10, now.month, now.day)
end = now
ticker = 'AAPL' #apple stock shortform

df = yf.download(ticker, start, end)
df['Moving_Average_100_days'] = df['Close'].rolling(100).mean()
df['Moving_Average_200_days'] = df['Close'].rolling(200).mean()

df['Percentage_changed'] = df.Close.pct_change()

data_training = pd.DataFrame(df.Close[0:int(len(df)*0.7)])
data_testing = pd.DataFrame(df.Close[int(len(df)*0.7):])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

data_training_array = scaler.fit_transform(data_training)

x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])


x_train, y_train = np.array(x_train), np.array(y_train)

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input

model = Sequential()
model.add(Input(shape=((100,1)))) 
model.add(LSTM(units=128, activation='tanh', return_sequences=True)) 
model.add(LSTM(units=64))
model.add(Dense(units=25)) 
model.add(Dense(units=1)) 

model.compile(optimizer='adam', loss='mean_squared_error' )
model.fit(x_train, y_train, epochs=50)

past_100_days = data_training.tail(100)

final_df = pd.concat([past_100_days, data_testing], ignore_index=True)  

input_data = scaler.fit_transform(final_df)

x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i,0])

x_test, y_test = np.array(x_test), np.array(y_test)
y_pred = model.predict(x_test)

y_pred = scaler.inverse_transform(y_pred.reshape(-1,1)).flatten()
y_test = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label='Original Price')
plt.plot(y_pred, 'r', label='Predicted Price')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()

